#Content-based Recommendation 

###Import packages and data


In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')
 

import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import re
import time
import tqdm as td

# Tutku path
#root_path ='/content/gdrive/MyDrive/MasterThesis/Data/KULrecommender'  #change dir to your project folder

#%cd root_path
#os.chdir(root_path)
#os.listdir()

In [2]:
!pip install plotly

     |████████████████████████████████| 15.3 MB 2.6 MB/s eta 0:00:01


In [6]:
df_books = pd.read_csv("book_summaries_final.csv", sep=',') # merged_info
#df_books = df_books.drop(['Unnamed: 0'], axis=1).reset_index()
df_books

,Unnamed: 0,level_0,index,exem_id,titelnr,isbn_x,book_title,deelaand,deeltitel,original_title,...,third_isbn_x,forth_isbn_x,Unnamed: 0.1,first_isbn_y,second_isbn_y,third_isbn_y,forth_isbn_y,book_summary_y,isbn_y,summary
0,0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,NaN,NaN,...,NaN,NaN,0.0,9782221243596,NaN,NaN,NaN,NaN,9782221243596,"""9 hackers combattent un dictateur. Des vies s..."
1,1,1,7,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,NaN,...,NaN,NaN,1.0,9789049807245,NaN,NaN,NaN,NaN,9789049807245,NaN
2,2,2,57,16826563,770735,9789028422087,Bekentenissen van een burger,NaN,NaN,Egy polgár vallomásai,...,NaN,NaN,2.0,9789028422087,NaN,NaN,NaN,NaN,9789028422087,Autobiografisch relaas van een jeugd voor de E...
3,3,3,72,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,NaN,NaN,Am Beispiel meines Bruders,...,9789056175627,9.789058e+12,3.0,9056175629,9057593211,9789056175627,9.789058e+12,NaN,9056175629,Autobiografisch getint relaas waarin een Duits...
4,4,4,82,16831632,1922287,9789403185705,De vreemdelinge,NaN,NaN,La straniera,...,NaN,NaN,4.0,9789403185705,NaN,NaN,NaN,NaN,9789403185705,Het levensverhaal van de auteur waarin ze besc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104665,3287452,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,NaN,NaN,...,NaN,NaN,98658.0,2849020664,9782849020661,NaN,NaN,NaN,2849020664,Découvrez la vie des mots : leur étymologie et...
104611,104611,104666,3287470,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,NaN,...,NaN,NaN,98659.0,9066484276,9789066484276,NaN,NaN,NaN,9066484276,NaN
104612,104612,104667,3287476,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,NaN,NaN,The mini visual dictionary,...,NaN,NaN,98660.0,9789066488427,NaN,NaN,NaN,NaN,9789066488427,NaN
104613,104613,104668,3287480,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,NaN,...,NaN,NaN,98661.0,9789460772832,NaN,NaN,NaN,NaN,9789460772832,NaN


In [7]:
# Read topic datafile
df_topics = pd.read_csv('KUL_topics.csv', sep="\t")
# Remove the unneccesary books which is not in the df_books
df_topics = df_topics[df_topics['titelnr'].isin(df_books['titelnr'])]
df_topics

,titelnr,topic_type,topic
0,4145849,genre,Ideeënliteratuur
1,4145849,genre,Romans
2,4145849,topic,Hacken
3,1536557,genre,Autobiografische literatuur
4,1536557,genre,Ontwikkelingsromans
...,...,...,...
2598250,576591,additional_topic,wereldatlas
2598251,576591,additional_topic,Aardrijkskunde ; atlassen
2598252,576591,additional_topic,Informatieve werken
2598253,576591,additional_topic,950.6 LEESZ


### Grouping genres, topics, additional topics

In [16]:
df = df_topics.drop_duplicates(['titelnr', 'topic_type', 'topic'])
# group the DataFrame by titelnr and type, and apply the join function to the value column
genres_df = df[df['topic_type'] == 'genre'].groupby('titelnr')['topic'].apply(', '.join).reset_index()
topics_df = df[df['topic_type'] == 'topic'].groupby('titelnr')['topic'].apply(', '.join).reset_index()
additional_topics_df = df[df['topic_type'] == 'additional_topic'].groupby('titelnr')['topic'].apply(', '.join).reset_index()

# merge the three DataFrames on the titelnr column
merged_df = pd.merge(genres_df, topics_df, on='titelnr', how='outer')
merged_df = pd.merge(merged_df, additional_topics_df, on='titelnr', how='outer')

# fill missing values with empty strings
merged_df.fillna('', inplace=True)

# rename columns
merged_df = merged_df.rename(columns={'topic_x': 'genre', 'topic_y': 'topic', 'topic' : 'additional_topic'})

# display the merged DataFrame
merged_df

,titelnr,genre,topic,additional_topic
0,53,Verhalen,,"verhalen, Cursiefjes"
1,58,Cursiefjes,,"Pensioen, Cursiefjes, Humoristische roman"
2,313,Verhalen,,"verhalen, VER, Verhalenbundels"
3,628,Romans,"Natuur, Obsessionele liefde, Platteland; Noorw...","liefde, Romans, Nobelprijs"
4,968,"Biografische literatuur, Historische literatuur",Rembrandt,"Rembrandt, biografieën, Romans, BIO, HIS, Remb..."
...,...,...,...,...
104050,3234265,,,"verhalen, levensvragen, obsessies, Psychologis..."
104051,3345229,,,"godsdiensten, vriendschap, liefde, Historisch,..."
104052,3358827,,,"symboliek, Kelten, Keltische kunst, 913.7, sym..."
104053,3409218,,,Fantasy (volw.)


In [17]:
df = merged_df

In [18]:
# merge columns into new column called "merged_info"
df['df_merged'] = df.apply(lambda row: f"{row['genre']} {row['additional_topic']} {row['topic']}", axis=1)

# display the DataFrame with the new column
df[['df_merged']]

,df_merged
0,"Verhalen verhalen, Cursiefjes"
1,"Cursiefjes Pensioen, Cursiefjes, Humoristische..."
2,"Verhalen verhalen, VER, Verhalenbundels"
3,"Romans liefde, Romans, Nobelprijs Natuur, Obse..."
4,"Biografische literatuur, Historische literatuu..."
...,...
104050,"verhalen, levensvragen, obsessies, Psychologi..."
104051,"godsdiensten, vriendschap, liefde, Historisch..."
104052,"symboliek, Kelten, Keltische kunst, 913.7, sy..."
104053,Fantasy (volw.)


In [19]:
# Define a function to remove duplicate words from a string
def remove_duplicates(text):
    # Split the text into a list of words
    words = text.split()
    # Create an empty set to keep track of unique words
    unique_words = set()
    # Create an empty list to hold the result words
    result_words = []
    # Loop through each word in the list of words
    for word in words:
        # If the word is not already in the set of unique words, add it to the set and the result list
        if word not in unique_words:
            unique_words.add(word)
            result_words.append(word)
    # Join the result words list back into a string
    result = " ".join(result_words)
    return result

# Apply the function to the 'text' column of the dataframe separately for each row
df['df_merged'] = df['df_merged'].apply(lambda x: remove_duplicates(x))
df

,titelnr,genre,topic,additional_topic,df_merged
0,53,Verhalen,,"verhalen, Cursiefjes","Verhalen verhalen, Cursiefjes"
1,58,Cursiefjes,,"Pensioen, Cursiefjes, Humoristische roman","Cursiefjes Pensioen, Cursiefjes, Humoristische..."
2,313,Verhalen,,"verhalen, VER, Verhalenbundels","Verhalen verhalen, VER, Verhalenbundels"
3,628,Romans,"Natuur, Obsessionele liefde, Platteland; Noorw...","liefde, Romans, Nobelprijs","Romans liefde, Romans, Nobelprijs Natuur, Obse..."
4,968,"Biografische literatuur, Historische literatuur",Rembrandt,"Rembrandt, biografieën, Romans, BIO, HIS, Remb...","Biografische literatuur, Historische literatuu..."
...,...,...,...,...,...
104050,3234265,,,"verhalen, levensvragen, obsessies, Psychologis...","verhalen, levensvragen, obsessies, Psychologis..."
104051,3345229,,,"godsdiensten, vriendschap, liefde, Historisch,...","godsdiensten, vriendschap, liefde, Historisch,..."
104052,3358827,,,"symboliek, Kelten, Keltische kunst, 913.7, sym...","symboliek, Kelten, Keltische kunst, 913.7, sym..."
104053,3409218,,,Fantasy (volw.),Fantasy (volw.)


In [20]:
df['df_merged'].iloc[2]

'Verhalen verhalen, VER, Verhalenbundels'

In [ ]:
# merge the second DataFrame into the first DataFrame
#f = pd.merge(df_books, merged_df, on='titelnr', how='left')
#df

### Creating text content for each book

In [21]:
# merge columns into new column called "merged_info"
df['merged_info'] = df.apply(lambda row: f"{row['df_merged']}" + ("" if pd.isna(row['book_summary']) or len(row['book_summary']) > 20 else f" {row['book_title']} by ") + f" {row['primary_author']} {row['language']} {'' if pd.isna(row['book_summary']) else row['book_summary']} " + 
                                (f" {row['awards']}" if not pd.isna(row['awards']) else ""), 
                            axis=1)

# display the DataFrame with the new column
df[['merged_info']]

KeyError: 'book_summary'

In [ ]:
df.to_csv('df_merged_info.csv')

In [ ]:
df2 = df

In [ ]:
df

,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,edition,...,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info,df_merged
0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,NaN,...,2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,,"Ideeënliteratuur, Romans Hacken Levy, Marc ""9 ...","Ideeënliteratuur, Romans Hacken"
1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,NaN,...,2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,NaN,...,2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsromans","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,NaN,...,2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Autobiografische literatuur autobiografieën, W...","Autobiografische literatuur autobiografieën, W..."
4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,NaN,...,2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans","Doofheid, Durastanti, Claudia","autobiografieën, Waargebeurd, doofheid, Autobi...","Autobiografische literatuur, Romans autobiogra...","Autobiografische literatuur, Romans autobiogra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",Nouv. éd.,...,2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,,Franse taal; woordenboeken,,"Franse taal; woordenboeken Robert, Paul Découv...",Franse taal; woordenboeken
104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,14de herz. uitg.,...,2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,,Nederlandse taal; woordenboeken,"Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w..."
104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",1,...,2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,..."
104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,1,...,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,,Nederlandse spelling,"woordenboeken, Nederlandse taal, spelling, Taa...","woordenboeken, Nederlands

In [ ]:
# add a new column with the length of each text string
df["text_length"] = df["merged_info"].apply(len)
df

In [ ]:
long_texts = df[df['text_length']>512]
long_texts

In [ ]:
ind_long_text = long_texts.index

### Summarizing Book Summaries over 200 characters

In [ ]:
# remove NaN values from the "text_column" column
df1 = df.dropna(subset=["book_summary"])

In [ ]:
# add a new column with the length of each text string
df1["summary_length"] = df1["book_summary"].apply(len)

<ipython-input-64-8f0f86524c4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["summary_length"] = df1["book_summary"].apply(len)


In [ ]:
df1['summary_length']

0         434
2         165
3         146
4         227
5          83
         ... 
104603     72
104604    136
104606     66
104607    860
104610    433
Name: summary_length, Length: 60724, dtype: int64

In [ ]:
long_summaries = pd.DataFrame(df.loc[ind_long_text, 'book_summary'])
long_summaries

,book_summary
2,Autobiografisch relaas van een jeugd voor de E...
4,Het levensverhaal van de auteur waarin ze besc...
6,"Paul Celan was een stateloze, Duitssprekende j..."
7,"Michael, Jos en Bart: drie generaties Demytten..."
11,Sarah is eind dertig en woont met haar gezin o...
...,...
104587,The summer holidays are dragging on and Harry ...
104588,Harry Potter is an ordinary boy who lives in a...
104589,When Harry and his best friends go back for th...
104591,Sixteen-year-old Katniss Everdeen regards it a...


In [ ]:
# remove NaN values from the "text_column" column
long_summaries = long_summaries.dropna(subset=["book_summary"])
# add a new column with the length of each text string
long_summaries["summary_length"] = long_summaries["book_summary"].apply(len)
long_summaries[long_summaries["summary_length"]>150]

,book_summary,summary_length
2,Autobiografisch relaas van een jeugd voor de E...,165
4,Het levensverhaal van de auteur waarin ze besc...,227
6,"Paul Celan was een stateloze, Duitssprekende j...",586
7,"Michael, Jos en Bart: drie generaties Demytten...",1068
11,Sarah is eind dertig en woont met haar gezin o...,1248
...,...,...
104587,The summer holidays are dragging on and Harry ...,554
104588,Harry Potter is an ordinary boy who lives in a...,913
104589,When Harry and his best friends go back for th...,396
104591,Sixteen-year-old Katniss Everdeen regards it a...,397


In [ ]:
long_summaries.to_csv("long_summaries.csv")

In [ ]:
!pip install torch

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners

In [ ]:
#!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define input and target text
input_text = "The quick brown fox jumps over the lazy dog."
target_text = "A quick brown fox jumps over a lazy dog."

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

# Generate summary
summary_ids = model.generate(input_ids,
                              num_beams=4,
                              max_length=150,
                              early_stopping=True)

# Decode summary
summary_text = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
print(summary_text)


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in t

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-89-1d850c5a83ed>", line 7, in <cell line: 7>
    model = T5ForConditionalGeneration.from_pretrained('t5-base')
  File "/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py", line 2174, in from_pretrained
    use_safetensors = kwargs.pop("use_safetensors", None if is_safetensors_available() else False)
  File "/usr/local/lib/python3.9/dist-packages/transformers/utils/import_utils.py", line 593, in is_safetensors_available
    return importlib.util.find_spec("safetensors") is not None
  File "/usr/lib/python3.9/importlib/util.py", line 103, in find_spec
    return _find_spec(fullname, parent_path)
  File "<frozen importlib._bootstrap>", line 925, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1423, in find_spec
  File "<frozen importlib.

### One example

In [ ]:
df

,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,edition,...,primary_author,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info
0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,NaN,...,"Levy, Marc",2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,,"Noa : roman by Levy, Marc (2022.0, Paris : Rob..."
1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,NaN,...,"Visser, Judith",2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...",Zondagskind : alsof opgroeien nog niet lastig ...
2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,NaN,...,"Márai, Sándor",2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Bekentenissen van een burger by Márai, Sándor ..."
3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,NaN,...,"Timm, Uwe",2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Mijn broer bijvoorbeeld by Timm, Uwe (2004.0, ..."
4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,NaN,...,"Durastanti, Claudia",2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans, Autobiogr...","Doofheid, Durastanti, Claudia, Doofheid, Duras...","autobiografieën, Waargebeurd, doofheid, Autobi...","De vreemdelinge by Durastanti, Claudia (2020.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,17003177,2135781,"9789045216102, 9789401615181",De make-over,Fix her up,De 23-jarige Georgie doet alles om de voormali...,"Uithoorn : Karakter Uitgevers B.V., © 2020",335 p.,NaN,...,"Bailey, Tessa",2020.0,DUT,ENG,de Bib Leuven Tweebronnen,B2,"Chicklit, Romans, Romantische literatuur, Chic...",,"Romantische literatuur, Chicklit, Romans, Tips...","De make-over by Bailey, Tessa (2020.0, Uithoor..."
1996,1996,17003543,1137459,9789049803988,Je bent geweldig,Three amazing things about you,NaN,"Amsterdam : Dwarsligger, cop. 2015",551 p. ; 12 cm,1,...,"Mansell, Jill",2015.0,DUT,ENG,de Bib Leuven Tweebronnen,B2,"Chicklit, Romans, Romantische literatuur, Chic...",,"Romantische literatuur, Vrouwen en liefde, Chi...","Je bent geweldig by Mansell, Jill (2015.0, Ams..."
1997,1997,17003681,1024955,9789046814628,Een zomer met Kim Bokaviet,NaN,Een jonge vrouw is obsessief verliefd op de vr...,"Amsterdam : Nieuw Amsterdam, cop. 2013",189 p.,1,...,"Toebak, Silvia",2013.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Chicklit, Romans, Romantische literatuur, Chic...",,"Romantische literatuur, Chicklit, Romans, CHI,...","Een zomer met Kim Bokaviet by Toebak, Silvia (..."
1998,1998,17004631,585081,"9069747790, 9789069747798",De kerstquilt,The Christmas quilt,Wanneer een oude vrouw voor het eerst na vijft...,"Amsterdam : Arena, cop. 2005",221 p.,NaN,...,"Chiaverini, Jennifer",2005.0,DUT,ENG,de Bib Leuven Tweebronnen,B2,"Chicklit, Romantische literatuur",,"vrouwen, Streek- en familieromans, Kerstmis, F...","De kerstquilt by Chiaverini, Jennifer (2005.0,..."


In [ ]:
df[df['book_title'] == "Keuken"] #1196477 - Alice in Wonderland
#560871 - Harry
#1152609 - nepal keuken

,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,edition,...,primary_author,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info
0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,NaN,...,"Levy, Marc",2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,,"Noa : roman by Levy, Marc (2022.0, Paris : Rob..."
1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,NaN,...,"Visser, Judith",2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...",Zondagskind : alsof opgroeien nog niet lastig ...
2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,NaN,...,"Márai, Sándor",2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Bekentenissen van een burger by Márai, Sándor ..."
3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,NaN,...,"Timm, Uwe",2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Mijn broer bijvoorbeeld by Timm, Uwe (2004.0, ..."
4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,NaN,...,"Durastanti, Claudia",2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans, Autobiogr...","Doofheid, Durastanti, Claudia, Doofheid, Duras...","autobiografieën, Waargebeurd, doofheid, Autobi...","De vreemdelinge by Durastanti, Claudia (2020.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",Nouv. éd.,...,"Robert, Paul",2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,,Franse taal; woordenboeken,,Le nouveau petit Robert : dictionnaire alphabé...
104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,14de herz. uitg.,...,"Den Boon, Ton",2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,,"Nederlandse taal; woordenboeken, Nederlandse t...","Nederlandse taal ; woordenboeken, Nederlands w...",Van Dale groot woordenboek van de Nederlandse ...
104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",1,...,"Corbeil, Jean-Claude",2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...",Van Dale beeldwoordenboek Nederlands-Turks by ...
104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,1,...,Nederlandse Taalunie,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,,"Nederlandse spelling, Nederlandse spelling, Ne...","woordenboeken, Nederlandse taal, spelling, Taa...",Het Groene Boekje : woordenlijst Nederlandse t...


In [ ]:
df

In [ ]:
df2[df2['titelnr']==1196477]['merged_info'].iloc[0] #770735 539177

'Alice in Wonderland by Adreani, Manuela (2016.0, Mechelen : Bakermat, 2016) - nan - Genre: Fantasieverhalen - Topic:  - Additional Topic: Sprookjes, Alice in Wonderland, Fantasieverhalen, fantasie, FAV, Sprookjesachtige verhalen, Juweeltjes (prentenboeken) - Language: DUT - Awards: nan'

In [ ]:
df2

In [ ]:
from transformers import AutoTokenizer, AutoModel

# load pre-trained multilingual model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# define two mixed-language texts
text1 = 'Harry Potter by Oostdijk, Andrea (2004.0, Groningen : Wolters-Noordhoff, 2004) - nan - Genre:  - Topic: Potter, Harry (personage) - Additional Topic: Harry Potter, Rowling, J.K., Over jeugdliteratuur, Rowling, J.K. ; Harry Potter, Rowling, J.K. (jeugd), Harry Potter (jeugd) - Language: DUT - Awards: nan'
text2 = 'Alice in Wonderland by Adreani, Manuela (2016.0, Mechelen : Bakermat, 2016) - nan - Genre: Fantasieverhalen - Topic:  - Additional Topic: Sprookjes, Alice in Wonderland, Fantasieverhalen, fantasie, FAV, Sprookjesachtige verhalen, Juweeltjes (prentenboeken) - Language: DUT - Awards: nan'

# encode the texts using the tokenizer and compute their similarity score using cosine similarity
inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
similarity_score = torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

print(similarity_score)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.7748420834541321


###Top-10 Similar books 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
 

import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import re
import time
import tqdm as td

# Tutku path
root_path ='/content/gdrive/MyDrive/MasterThesis/Data/KULrecommender'  #change dir to your project folder

# Edu path
#root_path = "/content/gdrive/MyDrive/data_thesis"

#%cd root_path
os.chdir(root_path)
os.listdir()

Mounted at /content/gdrive


['KUL_books.csv',
 'KUL_topics.csv',
 'KUL_authors.csv',
 'KUL_hist.csv',
 'KUL_hist_2.csv',
 'KUL_user.csv',
 'KUL_user_2.csv',
 'KUL_books_item_type_added.csv',
 'results_df_0_1000.csv',
 'results_df_1000_10000.csv',
 'results_df_15000_20000.csv',
 'results_df_20000_25000.csv',
 'results_df_25000_30000.csv',
 'results_df_40000_45000.csv',
 'results_df_45000_50000.csv',
 'results_df_50000_55000.csv',
 'results_df_50000_55000.gsheet',
 'results_df_55000_60000.csv',
 'results_df_60000_65000.csv',
 'results_df_65000_70000.csv',
 'results_df_70000_75000.csv',
 'results_df_75000_80000.csv',
 'results_df_80000_85000.csv',
 'results_df_85000_90000.csv',
 'results_df_10000_15000.csv',
 'results_df_35000_40000 (1).gsheet',
 'results_df_30000_35000 (1).gsheet',
 'results_df_35000_40000 (1).csv',
 'results_df_30000_35000.gsheet',
 'results_df_30000_35000.csv',
 'books_.csv',
 'results_df_35000_40000.gsheet',
 'results_df_35000_40000.csv',
 'results_df.csv',
 'results_df_15000_20000.gsheet',
 're

In [ ]:
df = pd.read_csv("df_merged_info.csv", sep=',')
df= df.drop(['Unnamed: 0'], axis=1).reset_index()
df

,level_0,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,...,primary_author,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info
0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,...,"Levy, Marc",2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,NaN,"Noa : roman by Levy, Marc (2022.0, Paris : Rob..."
1,1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,...,"Visser, Judith",2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...",Zondagskind : alsof opgroeien nog niet lastig ...
2,2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,...,"Márai, Sándor",2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Bekentenissen van een burger by Márai, Sándor ..."
3,3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,...,"Timm, Uwe",2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Mijn broer bijvoorbeeld by Timm, Uwe (2004.0, ..."
4,4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,...,"Durastanti, Claudia",2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans, Autobiogr...","Doofheid, Durastanti, Claudia, Doofheid, Duras...","autobiografieën, Waargebeurd, doofheid, Autobi...","De vreemdelinge by Durastanti, Claudia (2020.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",...,"Robert, Paul",2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,NaN,Franse taal; woordenboeken,NaN,Le nouveau petit Robert : dictionnaire alphabé...
104611,104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,...,"Den Boon, Ton",2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse taal; woordenboeken, Nederlandse t...","Nederlandse taal ; woordenboeken, Nederlands w...",Van Dale groot woordenboek van de Nederlandse ...
104612,104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",...,"Corbeil, Jean-Claude",2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,NaN,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...",Van Dale beeldwoordenboek Nederlands-Turks by ...
104613,104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,...,Nederlandse Taalunie,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse spelling, Nederlandse spelling, Ne...","woordenboeken, Nederlandse taal, spelling, Taa...",Het Groene Boekje : woordenlijst Nederlandse t...


In [ ]:
!pip install transformers
!pip install torch
!pip install scipy
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=565b16bf05ca050be5d4e278d235565b589581cff166e851e

In [ ]:
df1 = df[:]

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained multilingual model and tokenizer
model_name = 'distilbert-base-multilingual-cased' 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')  # move model to GPU

# Define function to get embeddings for a text
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to('cuda')
    outputs = model(**inputs).last_hidden_state.mean(dim=1)
    return outputs.cpu().detach().numpy()[0]

# Get the number of books
num_books = len(df)

# Initialize the matrix to store embeddings
embeddings_matrix = np.zeros((num_books, 768))

# Iterate over the DataFrame and get embeddings for each merged_info
for i, row in df.iterrows():
    embeddings_matrix[i] = get_embeddings(row['merged_info'])

embeddings_matrix

# Save the embeddings matrix as a .npy file
np.save('embeddings_matrix.npy', embeddings_matrix)

In [ ]:
# Load the embeddings matrix from the .npy file
embeddings_matrix = np.load('embeddings_matrix.npy')

In [ ]:
#Cosine similarities for one book 
import torch
import numpy as np
import random

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) 

# Convert embedding_matrix to a PyTorch tensor
embedding_tensor = torch.tensor(embeddings_matrix)

# Compute cosine similarities with the first book
first_book_embeddings = embedding_tensor[0]
cosine_similarities = torch.cosine_similarity(first_book_embeddings.unsqueeze(0), embedding_tensor, dim=1)

# Convert cosine_similarities tensor to a NumPy array
cosine_similarities = cosine_similarities.cpu().numpy()

# Assign cosine similarities to the 'cosine_similarities_with_first_book' column of the DataFrame
df['cosine_similarities_with_first_book'] = cosine_similarities


In [ ]:
df

,level_0,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,...,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info,cosine_similarities_with_first_book
0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,...,2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,NaN,"Noa : roman by Levy, Marc (2022.0, Paris : Rob...",1.000000
1,1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,...,2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...",Zondagskind : alsof opgroeien nog niet lastig ...,0.757144
2,2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,...,2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Bekentenissen van een burger by Márai, Sándor ...",0.752963
3,3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,...,2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Mijn broer bijvoorbeeld by Timm, Uwe (2004.0, ...",0.774742
4,4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,...,2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans, Autobiogr...","Doofheid, Durastanti, Claudia, Doofheid, Duras...","autobiografieën, Waargebeurd, doofheid, Autobi...","De vreemdelinge by Durastanti, Claudia (2020.0...",0.753849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",...,2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,NaN,Franse taal; woordenboeken,NaN,Le nouveau petit Robert : dictionnaire alphabé...,0.910284
104611,104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,...,2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse taal; woordenboeken, Nederlandse t...","Nederlandse taal ; woordenboeken, Nederlands w...",Van Dale groot woordenboek van de Nederlandse ...,0.664141
104612,104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",...,2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,NaN,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...",Van Dale beeldwoordenboek Nederlands-Turks by ...,0.753009
104613,104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,...,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse spelling, Nederlandse spelling, Ne...","woordenboeken, Nederlandse taal, spelling, Taa...",Het Groene Boekje : woordenlijst Nederlandse t...,0.691398


In [ ]:
seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) 

# Load pre-trained multilingual model and tokenizer
model_name = 'distilbert-base-multilingual-cased' #
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')  # move model to GPU

text1 = "Bekentenissen van een burger by Márai, Sándor (2007.0, Amsterdam : Wereldbibliotheek, cop. 2007) - Autobiografisch relaas van een jeugd voor de Eerste Wereldoorlog in het hoogontwikkelde Groot-Hongaarse Rijk en van verblijven daarna in Frankfurt, Parijs en Londen. - Genre: Autobiografische literatuur, Ontwikkelingsromans, Autobiografische literatuur, Ontwikkelingsromans, Autobiografische literatuur, Ontwikkelingsromans, Autobiografische literatuur, Ontwikkelingsromans - Topic: Europa; 1900-1945, Oostenrijks-Hongaarse monarchie, Márai, Sándor, Europa; 1900-1945, Oostenrijks-Hongaarse monarchie, Márai, Sándor, Europa; 1900-1945, Oostenrijks-Hongaarse monarchie, Márai, Sándor, Europa; 1900-1945, Oostenrijks-Hongaarse monarchie, Márai, Sándor - Additional Topic: Europa, 1900 - 1945, Autobiografische literatuur, 1940 - 1945, Ontwikkelingsromans, Oostenrijks-Hongaarse monarchie, AUT, Hongaarse letterkunde, Humo, De Standaard der Letteren, De Morgen, Uitgelezen, Europa ; 1940-1945-, Oostenrijks-Hongaarse monarchie (volwassenen fictie), Ontwikkelingsromans (volwassenen fictie), Europa (volwassenen fictie), Oostenrijks-Hongaarse monarchie (volwass. fictie), Europa ; 1900-1945 (volwass. fictie), Historische romans (volw.), Europa (volwassenen verhalend), Oostenrijks-Hongaarse monarchie (volwassenen verhalend), Europa, 1900 - 1945, Autobiografische literatuur, 1940 - 1945, Ontwikkelingsromans, Oostenrijks-Hongaarse monarchie, AUT, Hongaarse letterkunde, Humo, De Standaard der Letteren, De Morgen, Uitgelezen, Europa ; 1940-1945-, Oostenrijks-Hongaarse monarchie (volwassenen fictie), Ontwikkelingsromans (volwassenen fictie), Europa (volwassenen fictie), Oostenrijks-Hongaarse monarchie (volwass. fictie), Europa ; 1900-1945 (volwass. fictie), Historische romans (volw.), Europa (volwassenen verhalend), Oostenrijks-Hongaarse monarchie (volwassenen verhalend), Europa, 1900 - 1945, Autobiografische literatuur, 1940 - 1945, Ontwikkelingsromans, Oostenrijks-Hongaarse monarchie, AUT, Hongaarse letterkunde, Humo, De Standaard der Letteren, De Morgen, Uitgelezen, Europa ; 1940-1945-, Oostenrijks-Hongaarse monarchie (volwassenen fictie), Ontwikkelingsromans (volwassenen fictie), Europa (volwassenen fictie), Oostenrijks-Hongaarse monarchie (volwass. fictie), Europa ; 1900-1945 (volwass. fictie), Historische romans (volw.), Europa (volwassenen verhalend), Oostenrijks-Hongaarse monarchie (volwassenen verhalend), Europa, 1900 - 1945, Autobiografische literatuur, 1940 - 1945, Ontwikkelingsromans, Oostenrijks-Hongaarse monarchie, AUT, Hongaarse letterkunde, Humo, De Standaard der Letteren, De Morgen, Uitgelezen, Europa ; 1940-1945-, Oostenrijks-Hongaarse monarchie (volwassenen fictie), Ontwikkelingsromans (volwassenen fictie), Europa (volwassenen fictie), Oostenrijks-Hongaarse monarchie (volwass. fictie), Europa ; 1900-1945 (volwass. fictie), Historische romans (volw.), Europa (volwassenen verhalend), Oostenrijks-Hongaarse monarchie (volwassenen verhalend) - Language: DUT - Awards: nan"
text2 = "Testament van de jeugd by Brittain, Vera (2016.0, Amsterdam : De Bezige Bij, 2016) - u.a. 20 Filmfotos (17,5x22 cm s/w), überw. UFA-Abzüge, 14 Fotos d. Ausstellung (12x17 cm s/w), 'Paul Wegener' zum 65. Geburtstag, Schillertheater Berlin, 1939, Dr., 16 S., zahlr. Ill. - Genre: Autobiografische literatuur, Historische literatuur, Romans, Verfilmde boeken - Topic: Groot-Brittannië; 1900-, Wereldoorlog I, Brittain, Vera - Additional Topic: autobiografieën, Historische literatuur, Groot-Brittannië, Wereldoorlog I, Verfilmde boeken, Autobiografische literatuur, Waar gebeurd, Romans, HIS, AUT, 4 sterren (Standaard der Letteren), Getuigenissen ; Wereldoorlog I, Vrouwelijke verpleegkundigen ; Groot-Brittannië ; Wereldoorlog I, Brits leger ; persoonlijke verhalen ; Wereldoorlog I, Brits leger ; medische diensten ; Wereldoorlog I, Dagboeken ; Britse verpleegsters ; Wereldoorlog I, Het beste van '14-'18 (Standaard der Letteren), 1900-, De Standaard der Letteren, De Morgen, Boeken, Wereldoorlog I (volw. verhalend), Brittain, Vera (volw. verhalend), Groot-Brittannië ; 1900- (volw. verhalend), Wereldoorlog I (volwassenen fictie), Brittain, Vera (volwassenen fictie), Groot-Brittannië ; 1900- (volwassenen fictie), Wereldoorlog I (volwass. fictie), Brittain, Vera (volwass. fictie), Groot-Brittannië ; 1900- (volwass. fictie), Verfilmde boeken (volw.), Wereldoorlog I (volw.), Autobiografische literatuur (volw.), Wereldoorlog I (volwassenen verhalend), Groot-Brittannië ; 1900- (volwassenen verhalend) - Language: DUT - Awards: nan"

inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True).to('cuda')  # move inputs to GPU
inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True).to('cuda')  # move inputs to GPU
outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: ignored

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from torch.cuda.amp import autocast, GradScaler
import random 

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) 

# Load pre-trained multilingual model and tokenizer
model_name = 'distilbert-base-multilingual-cased' 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')  # move model to GPU

# Define function to compute similarity score
def get_similarity_score(embedding1, embedding2):
    with torch.no_grad():
        return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=1).item()

# Function to create embedding matrix for all books in the dataset
@autocast()
def create_embedding_matrix(df, batch_size=16, max_length=512):
    embeddings = []
    for i in range(0, len(df), batch_size):
        batch = df['merged_info'].iloc[i:i+batch_size].tolist()
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to('cuda')
        outputs = model(**inputs).last_hidden_state.mean(dim=1)
        embeddings.append(outputs)
    embeddings = torch.cat(embeddings)
    embeddings = embeddings.float()
    return embeddings

# Function to find top-10 similar books
def find_similar_books(df, titelnr_list):
    # Create embedding matrix for all books in the dataset
    embeddings = create_embedding_matrix(df)
    
    # Get embeddings for specified books
    inputs = tokenizer(df.loc[df['titelnr'].isin(titelnr_list), 'merged_info'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
    query_embeddings = model(**inputs).last_hidden_state.mean(dim=1).float()
    
    # Compute the similarity scores between the specified books and all other books
    similarity_scores = torch.nn.functional.cosine_similarity(query_embeddings, embeddings, dim=1).tolist()
    
    # Filter out the specified books and sort by similarity score
    similar_books = df.loc[~df['titelnr'].isin(titelnr_list), ['titelnr', 'book_title', 'merged_info']].assign(similarity_score=similarity_scores).sort_values(by='similarity_score', ascending=False).head(10)
    return similar_books

# Example usage: find top-10 books similar to books with titelnr 770735 and 539177
#df = pd.read_csv('books.csv')
similar_books = find_similar_books(df1, [1196477])
similar_books


Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: ignored

In [ ]:
#105,000 comparison took 30 mins (WORKS BEST SO FAR)
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import random 

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) 

# Load pre-trained multilingual model and tokenizer
model_name = 'distilbert-base-multilingual-cased' #
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')  # move model to GPU

# Define function to compute similarity score
def get_similarity_score(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True).to('cuda')  # move inputs to GPU
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True).to('cuda')  # move inputs to GPU
    outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
    outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
    return torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

# Function to find top-10 similar books
def find_similar_books(df, titelnr_list):
    # Combine the text of the specified books
    combined_text = ' '.join(df.loc[df['titelnr'].isin(titelnr_list), 'merged_info'])
    # Compute the similarity scores between the combined text and all other books
    df['similarity_score'] = df['merged_info'].apply(lambda x: get_similarity_score(x, combined_text))
    # Filter out the specified books and sort by similarity score
    similar_books = df.loc[~df['titelnr'].isin(titelnr_list), ['titelnr', 'book_title', 'merged_info', 'similarity_score']].sort_values(by='similarity_score', ascending=False).head(10)
    return similar_books

# Example usage: find top-10 books similar to books with titelnr 770735 and 539177
#df = pd.read_csv('books.csv')
similar_books = find_similar_books(df1, [1196477])
similar_books


Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: ignored

In [ ]:
df

,level_0,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,...,primary_author,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info
0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,...,"Levy, Marc",2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,NaN,"Noa : roman by Levy, Marc (2022.0, Paris : Rob..."
1,1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,...,"Visser, Judith",2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...",Zondagskind : alsof opgroeien nog niet lastig ...
2,2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,...,"Márai, Sándor",2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Bekentenissen van een burger by Márai, Sándor ..."
3,3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,...,"Timm, Uwe",2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Mijn broer bijvoorbeeld by Timm, Uwe (2004.0, ..."
4,4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,...,"Durastanti, Claudia",2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans, Autobiogr...","Doofheid, Durastanti, Claudia, Doofheid, Duras...","autobiografieën, Waargebeurd, doofheid, Autobi...","De vreemdelinge by Durastanti, Claudia (2020.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",...,"Robert, Paul",2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,NaN,Franse taal; woordenboeken,NaN,Le nouveau petit Robert : dictionnaire alphabé...
104611,104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,...,"Den Boon, Ton",2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse taal; woordenboeken, Nederlandse t...","Nederlandse taal ; woordenboeken, Nederlands w...",Van Dale groot woordenboek van de Nederlandse ...
104612,104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",...,"Corbeil, Jean-Claude",2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,NaN,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...",Van Dale beeldwoordenboek Nederlands-Turks by ...
104613,104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,...,Nederlandse Taalunie,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,"Nederlandse spelling, Nederlandse spelling, Ne...","woordenboeken, Nederlandse taal, spelling, Taa...",Het Groene Boekje : woordenlijst Nederlandse t...


In [ ]:
#1000 books takes 42 mins with TPU otherwise 15 mins (larger package)
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained multilingual model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define function to compute similarity score
def get_similarity_score(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
    outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
    outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
    return torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

# Function to find top-10 similar books
def find_similar_books(df, titelnr_list):
    # Combine the text of the specified books
    combined_text = ' '.join(df.loc[df['titelnr'].isin(titelnr_list), 'merged_info'])
    # Compute the similarity scores between the combined text and all other books
    df['similarity_score'] = df['merged_info'].apply(lambda x: get_similarity_score(x, combined_text))
    # Filter out the specified books and sort by similarity score
    similar_books = df.loc[~df['titelnr'].isin(titelnr_list), ['titelnr', 'book_title', 'merged_info', 'similarity_score']].sort_values(by='similarity_score', ascending=False).head(10)
    return similar_books

# Example usage: find top-10 books similar to books with titelnr 770735 and 539177
similar_books = find_similar_books(df, [1196477])
similar_books


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-99402b31fa7e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,titelnr,book_title,merged_info,similarity_score
85,413839,Alles is ijdelheid,"Alles is ijdelheid by Goll, Claire (1999.0, Am...",0.235950
96,408688,"Een hiernamaals : opstellen, in memoriams","Een hiernamaals : opstellen, in memoriams by B...",0.217222
21,1294088,Swanns kant op,"Swanns kant op by Proust, Marcel (2018.0, Amst...",0.209250
95,408655,Mijn moeder zong,"Mijn moeder zong by Queneau, Raymond (1999.0, ...",0.203334
99,423574,De fakkel in het oor : mijn levensgeschiedenis...,De fakkel in het oor : mijn levensgeschiedenis...,0.197611
89,1583499,Het keerpunt : een autobiografie,"Het keerpunt : een autobiografie by Mann, Klau...",0.196149
65,1155867,Trouwboekje,"Trouwboekje by Modiano, Patrick (2015.0, Amste...",0.177628
25,1125187,Het duister zichtbaar : verslag van een voorbi...,Het duister zichtbaar : verslag van een voorbi...,0.167206
34,636885,De blik naar buiten,"De blik naar buiten by Ernaux, Annie (2007.0, ...",0.159056
84,603846,Dorrestijns vogelgids,"Dorrestijns vogelgids by Dorrestijn, Hans (200...",0.158577


In [ ]:
# in batches but doesnt add much
df = df[:2000]
#In batches
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained multilingual model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define function to compute similarity score
def get_similarity_score(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
    outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
    outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
    return torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

# Function to find top-10 similar books
def find_similar_books(df, titelnr_list):
    # Initialize empty DataFrame to store results
    similar_books = pd.DataFrame(columns=['titelnr', 'book_title', 'merged_info', 'similarity_score'])
    # Compute similarity scores in batches of 100
    for i in range(0, len(df), 100):
        batch_df = df.iloc[i:i+100]
        # Combine the text of the specified books
        combined_text = ' '.join(batch_df.loc[batch_df['titelnr'].isin(titelnr_list), 'merged_info'])
        # Compute the similarity scores between the combined text and all other books in the batch
        batch_df['similarity_score'] = batch_df['merged_info'].apply(lambda x: get_similarity_score(x, combined_text))
        # Filter out the specified books and sort by similarity score
        batch_similar_books = batch_df.loc[~batch_df['titelnr'].isin(titelnr_list), ['titelnr', 'book_title', 'merged_info', 'similarity_score']].sort_values(by='similarity_score', ascending=False).head(10)
        # Append the batch results to the overall results
        similar_books = pd.concat([similar_books, batch_similar_books])
    return similar_books

# Example usage: find top-10 books similar to books with titelnr 1196477
#df = pd.read_csv('books.csv')
similar_books = find_similar_books(df, [1196477])
similar_books


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-26-29862786c7e0>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,titelnr,book_title,merged_info,similarity_score
85,413839,Alles is ijdelheid,"Alles is ijdelheid by Goll, Claire (1999.0, Am...",0.235950
96,408688,"Een hiernamaals : opstellen, in memoriams","Een hiernamaals : opstellen, in memoriams by B...",0.217222
21,1294088,Swanns kant op,"Swanns kant op by Proust, Marcel (2018.0, Amst...",0.209250
95,408655,Mijn moeder zong,"Mijn moeder zong by Queneau, Raymond (1999.0, ...",0.203334
99,423574,De fakkel in het oor : mijn levensgeschiedenis...,De fakkel in het oor : mijn levensgeschiedenis...,0.197611
...,...,...,...,...
1946,1288808,De flirtcursus,"De flirtcursus by De Koning, Laura (2018.0, Am...",0.217042
1918,1154265,Zin : mijn maand met Q,"Zin : mijn maand met Q by Van Laren, Jackie (2...",0.214843
1978,1290593,Duingras,"Duingras by Van Laren, Jackie (2018.0, Amsterd...",0.214329
1938,1238188,Als mode vreemdgaat,"Als mode vreemdgaat by Friedrich, Anna (2017.0...",0.211308


### Creating Similarity Matrix 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
 

import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import re
import time
import tqdm as td

# Tutku path
root_path ='/content/gdrive/MyDrive/MasterThesis/Data/KULrecommender'  #change dir to your project folder

# Edu path
#root_path = "/content/gdrive/MyDrive/data_thesis"

#%cd root_path
os.chdir(root_path)
os.listdir()


Mounted at /content/gdrive


['KUL_books.csv',
 'KUL_topics.csv',
 'KUL_authors.csv',
 'KUL_hist.csv',
 'KUL_hist_2.csv',
 'KUL_user.csv',
 'KUL_user_2.csv',
 'KUL_books_item_type_added.csv',
 'results_df_0_1000.csv',
 'results_df_1000_10000.csv',
 'results_df_15000_20000.csv',
 'results_df_20000_25000.csv',
 'results_df_25000_30000.csv',
 'results_df_40000_45000.csv',
 'results_df_45000_50000.csv',
 'results_df_50000_55000.csv',
 'results_df_50000_55000.gsheet',
 'results_df_55000_60000.csv',
 'results_df_60000_65000.csv',
 'results_df_65000_70000.csv',
 'results_df_70000_75000.csv',
 'results_df_75000_80000.csv',
 'results_df_80000_85000.csv',
 'results_df_85000_90000.csv',
 'results_df_10000_15000.csv',
 'results_df_35000_40000 (1).gsheet',
 'results_df_30000_35000 (1).gsheet',
 'results_df_35000_40000 (1).csv',
 'results_df_30000_35000.gsheet',
 'results_df_30000_35000.csv',
 'books_.csv',
 'results_df_35000_40000.gsheet',
 'results_df_35000_40000.csv',
 'results_df.csv',
 'results_df_15000_20000.gsheet',
 're

In [ ]:
!pip install transformers
!pip install torch
!pip install scipy
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 30.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=2bd6621d9c1b91f925e94b638e658597df6176250e5a3b0

In [ ]:
df = pd.read_csv("df_merged_info.csv", sep=',')
df= df.drop(['Unnamed: 0'], axis=1).reset_index()
df

,level_0,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,...,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info,df_merged
0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,...,2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,NaN,"Ideeënliteratuur, Romans Hacken Levy, Marc ""9 ...","Ideeënliteratuur, Romans Hacken"
1,1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,...,2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
2,2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,...,2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsromans","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
3,3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,...,2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Autobiografische literatuur autobiografieën, W...","Autobiografische literatuur autobiografieën, W..."
4,4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,...,2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans","Doofheid, Durastanti, Claudia","autobiografieën, Waargebeurd, doofheid, Autobi...","Autobiografische literatuur, Romans autobiogra...","Autobiografische literatuur, Romans autobiogra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",...,2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,NaN,Franse taal; woordenboeken,NaN,"Franse taal; woordenboeken Robert, Paul Découv...",Franse taal; woordenboeken
104611,104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,...,2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,Nederlandse taal; woordenboeken,"Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w..."
104612,104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",...,2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,NaN,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,..."
104613,104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,...,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,Nederlandse spelling,"woordenboeken, Nederlandse taal, spelling, Taa...","woordenboeken, Neder

In [ ]:
#This computes embeddings_matrix using distilbert-base-multilingual-cased model.
#You can use the output of this cell as the input to cosine similarity 
#calculation (took 17mins to run)
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import random 

seed_value = 42

random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value) 

# Load pre-trained multilingual model and tokenizer
model_name = 'distilbert-base-multilingual-cased' 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cuda')  # move model to GPU

# Define function to get embeddings for a text
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to('cuda')
    outputs = model(**inputs).last_hidden_state.mean(dim=1)
    return outputs.cpu().detach().numpy()[0]

# Get the number of books
num_books = len(df)

# Initialize the matrix to store embeddings
embeddings_matrix = np.zeros((num_books, 768))

# Iterate over the DataFrame and get embeddings for each merged_info
for i, row in df.iterrows():
    embeddings_matrix[i] = get_embeddings(row['merged_info'])

embeddings_matrix

In [ ]:
# Save the embeddings matrix as a .npy file
np.save('embeddings_matrix.npy', embeddings_matrix)

In [ ]:
# Load the embeddings matrix from the .npy file
embeddings_matrix = np.load('embeddings_matrix.npy')

In [ ]:
# save the array as a csv file
np.savetxt('embeddings_matrix.csv', embeddings_matrix, delimiter=',')

#### Cosine similarity

In [ ]:
# Calculating cosine similarities with first book for all the books using 
#embeddings_matrix (took 4secs to run)
import torch
import numpy as np

# Convert embedding_matrix to a PyTorch tensor
embedding_tensor = torch.tensor(embeddings_matrix)

# Compute cosine similarities with the first book
first_book_embeddings = embedding_tensor[0]
cosine_similarities = torch.cosine_similarity(first_book_embeddings.unsqueeze(0), embedding_tensor, dim=1)

# Convert cosine_similarities tensor to a NumPy array
cosine_similarities = cosine_similarities.cpu().numpy()

# Assign cosine similarities to the 'cosine_similarities_with_first_book' column of the DataFrame
df['cosine_similarities_with_first_book'] = cosine_similarities


In [ ]:
cosine_similarities_matrix

In [ ]:
cosine_similarities_matrix.to_csv('cosine_similarities_matrix2.csv')

In [ ]:
def find_similar_books(df, titelnr_list, batch_size=1000):
    num_books = len(df)
    similar_books = pd.DataFrame(columns=['titelnr', 'book_title', 'merged_info', 'similarity_score'])
    
    # Loop through data frame in batches
    for i in range(0, num_books, batch_size):
        batch = df.iloc[i:i+batch_size]
        
        # Combine the text of the specified books
        combined_text = ' '.join(batch.loc[batch['titelnr'].isin(titelnr_list), 'merged_info'])
        
        # Compute the similarity scores between the combined text and all other books in the batch
        batch['similarity_score'] = batch['merged_info'].apply(lambda x: get_similarity_score(x, combined_text))
        
        # Filter out the specified books and sort by similarity score
        batch_similar_books = batch.loc[~batch['titelnr'].isin(titelnr_list), ['titelnr', 'book_title', 'merged_info', 'similarity_score']].sort_values(by='similarity_score', ascending=False).head(10)
        
        # Append the top similar books from this batch to the overall list
        similar_books = similar_books.append(batch_similar_books)
    
    # Sort the final list by similarity score and return the top-10
    similar_books = similar_books.sort_values(by='similarity_score', ascending=False).head(10)
    return similar_books


In [ ]:
similar_books = find_similar_books(df, [770735, 539177], batch_size=1000)
similar_books

KeyboardInterrupt: ignored

In [ ]:
similar_books

In [ ]:
import multiprocessing as mp
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.sparse import dok_matrix

# load pre-trained multilingual model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


# define function to compute similarity score
def get_similarity_score(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
    outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
    outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
    return torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

df = df2
# define function to compute similarity score
def compute_similarity(i, j):
    return (i, j, get_similarity_score(df['merged_info'][i], df['merged_info'][j]))

# create sparse similarity matrix
num_books = len(df)
similarity_matrix = dok_matrix((num_books, num_books))
with mp.Pool(processes=4) as pool: # set number of processes to use
    results = [pool.apply_async(compute_similarity, args=(i, j)) for i in range(num_books) for j in range(i+1, num_books)]
    for r in results:
        i, j, sim = r.get()
        similarity_matrix[i, j] = sim
        similarity_matrix[j, i] = sim

In [ ]:
similarity_matrix.to_array()

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.sparse import dok_matrix

# load pre-trained multilingual model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


# define function to compute similarity score
def get_similarity_score(text1, text2):
    inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)
    outputs1 = model(**inputs1).last_hidden_state.mean(dim=1)
    outputs2 = model(**inputs2).last_hidden_state.mean(dim=1)
    return torch.nn.functional.cosine_similarity(outputs1, outputs2, dim=1).item()

# create sparse similarity matrix
num_books = len(df)
similarity_matrix = dok_matrix((num_books, num_books))
'''for i in range(num_books):
    for j in range(1):
        similarity_matrix[i, j] = get_similarity_score(df['merged_info'][i], df['merged_info'][j])
    print(i)
'''
similarity_matrix.toarray()


In [ ]:
similarity_matrix.to_csv("similarity_matrix.csv")

### Cosine Similarity Matrix 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
 

import os
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import re
import time
import tqdm as td

# Tutku path
root_path ='/content/gdrive/MyDrive/MasterThesis/Data/KULrecommender'  #change dir to your project folder

# Edu path
#root_path = "/content/gdrive/MyDrive/data_thesis"

#%cd root_path
os.chdir(root_path)
os.listdir()


!pip install transformers
!pip install torch
!pip install scipy
!pip install sentence_transformers

df = pd.read_csv("df_merged_info.csv", sep=',')
df= df.drop(['Unnamed: 0'], axis=1).reset_index()
df

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4f26611f3ec669d3b96035

,level_0,index,exem_id,titelnr,isbn,book_title,original_title,book_summary,publisher,collation,...,publication_date,language,original_language,location,item_type,genre,topic,additional_topic,merged_info,df_merged
0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,"""9 hackers combattent un dictateur. Des vies s...","Paris : Robert Laffont/Versilio, 2022",367 p. : ill.,...,2022.0,FRE,NaN,de Bib Leuven Tweebronnen,B2,"Ideeënliteratuur, Romans",Hacken,NaN,"Ideeënliteratuur, Romans Hacken Levy, Marc ""9 ...","Ideeënliteratuur, Romans Hacken"
1,1,1,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,"[Amsterdam] : Dwarsligger, © 2019",971 p. ; 12 cm,...,2019.0,DUT,NaN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsroma...","Autismespectrumstoornissen, Tijdsbeeld; 1980-1...","Autisme, Aspergersyndroom, Ontwikkelingsromans...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
2,2,2,16826563,770735,9789028422087,Bekentenissen van een burger,Egy polgár vallomásai,Autobiografisch relaas van een jeugd voor de E...,"Amsterdam : Wereldbibliotheek, cop. 2007",463 p.,...,2007.0,DUT,HUN,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Ontwikkelingsromans","Europa; 1900-1945, Oostenrijks-Hongaarse monar...","Europa, 1900 - 1945, Autobiografische literatu...","Autobiografische literatuur, Ontwikkelingsroma...","Autobiografische literatuur, Ontwikkelingsroma..."
3,3,3,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,Am Beispiel meines Bruders,Autobiografisch getint relaas waarin een Duits...,"Amsterdam : Podium, 2004",151 p.,...,2004.0,DUT,GER,de Bib Leuven Tweebronnen,B2,Autobiografische literatuur,"Kinderen van nazi's, Schuldvraag; Wereldoorlog...","autobiografieën, Wereldoorlog II, Autobiografi...","Autobiografische literatuur autobiografieën, W...","Autobiografische literatuur autobiografieën, W..."
4,4,4,16831632,1922287,9789403185705,De vreemdelinge,La straniera,Het levensverhaal van de auteur waarin ze besc...,"Amsterdam : De Bezige Bij, 2020",284 p.,...,2020.0,DUT,ITA,de Bib Leuven Tweebronnen,B2,"Autobiografische literatuur, Romans","Doofheid, Durastanti, Claudia","autobiografieën, Waargebeurd, doofheid, Autobi...","Autobiografische literatuur, Romans autobiogra...","Autobiografische literatuur, Romans autobiogra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104610,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,Découvrez la vie des mots : leur étymologie et...,"Paris : Dictionnaires Le Robert, 2005","XXXVII, 2949 p.",...,2005.0,FRE,NaN,Hulpgevangenis Leuven,ZZ,NaN,Franse taal; woordenboeken,NaN,"Franse taal; woordenboeken Robert, Paul Découv...",Franse taal; woordenboeken
104611,104611,104611,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,"Utrecht : Van Dale Lexicografie, 2005",3 dl. + cd-rom,...,2005.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,Nederlandse taal; woordenboeken,"Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w...","Nederlandse taal ; woordenboeken, Nederlands w..."
104612,104612,104612,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,The mini visual dictionary,NaN,"Utrecht : Van Dale, 2010","VI, 598 p. : ill.",...,2010.0,DUT,ENG,Hulpgevangenis Leuven,ZZ,NaN,Turkse taal; beeldwoordenboeken,"Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,...","Nederlandse taal voor anderstaligen, Taalpunt,..."
104613,104613,104613,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,"Utrecht : Van Dale, 2015",1207 p.,...,2015.0,DUT,NaN,Hulpgevangenis Leuven,ZZ,NaN,Nederlandse spelling,"woordenboeken, Nederlandse taal, spelling, Taa...","woordenboeken, Neder

In [243]:
from google.colab import drive
drive.mount('/content/gdrive')

os.chdir(root_path)
os.listdir()
 
df = pd.read_csv("df_merged_info2.csv", sep=',')
#df= df.drop(['Unnamed: 0.1'], axis=1).reset_index()
df

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-243-1857b516ca65>:7: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df_merged_info2.csv", sep=',')


,Unnamed: 0.1,level_0,index,exem_id,titelnr,isbn_x,book_title,deelaand,deeltitel,original_title,...,third_isbn_x,forth_isbn_x,Unnamed: 0,first_isbn_y,second_isbn_y,third_isbn_y,forth_isbn_y,book_summary_y,isbn_y,summary
0,0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,NaN,NaN,...,NaN,NaN,0.0,9782221243596,NaN,NaN,NaN,NaN,9782221243596,"""9 hackers combattent un dictateur. Des vies s..."
1,1,1,7,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,NaN,...,NaN,NaN,1.0,9789049807245,NaN,NaN,NaN,NaN,9789049807245,NaN
2,2,2,57,16826563,770735,9789028422087,Bekentenissen van een burger,NaN,NaN,Egy polgár vallomásai,...,NaN,NaN,2.0,9789028422087,NaN,NaN,NaN,NaN,9789028422087,Autobiografisch relaas van een jeugd voor de E...
3,3,3,72,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,NaN,NaN,Am Beispiel meines Bruders,...,9789056175627,9.789058e+12,3.0,9056175629,9057593211,9789056175627,9.789058e+12,NaN,9056175629,Autobiografisch getint relaas waarin een Duits...
4,4,4,82,16831632,1922287,9789403185705,De vreemdelinge,NaN,NaN,La straniera,...,NaN,NaN,4.0,9789403185705,NaN,NaN,NaN,NaN,9789403185705,Het levensverhaal van de auteur waarin ze besc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104665,3287452,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,NaN,NaN,...,NaN,NaN,98658.0,2849020664,9782849020661,NaN,NaN,NaN,2849020664,Découvrez la vie des mots : leur étymologie et...
104611,104611,104666,3287470,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,NaN,...,NaN,NaN,98659.0,9066484276,9789066484276,NaN,NaN,NaN,9066484276,NaN
104612,104612,104667,3287476,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,NaN,NaN,The mini visual dictionary,...,NaN,NaN,98660.0,9789066488427,NaN,NaN,NaN,NaN,9789066488427,NaN
104613,104613,104668,3287480,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,NaN,...,NaN,NaN,98661.0,9789460772832,NaN,NaN,NaN,NaN,9789460772832,NaN


In [245]:
df[df['isbn_x'].str.contains('9789021563855')]['book_title']

41449    Miracle morning : 6 gewoontes om je leven succ...
Name: book_title, dtype: object

In [246]:
#Assign title_author_id
# concatenate the two series horizontally with comma-separated values
concatenated_series = pd.concat([df['book_title'], df['primary_author']], axis=1)
df['title_author'] = concatenated_series['book_title'].str.cat(concatenated_series['primary_author'], sep=',')
df['book_title_c'] = df['title_author'].astype("category")
df['book_title_id'] = df['book_title_c'].cat.codes
df['isbn'] = df['isbn_x']
df

,Unnamed: 0.1,level_0,index,exem_id,titelnr,isbn_x,book_title,deelaand,deeltitel,original_title,...,second_isbn_y,third_isbn_y,forth_isbn_y,book_summary_y,isbn_y,summary,title_author,book_title_c,book_title_id,isbn
0,0,0,0,24442024,4145849,9782221243596,Noa : roman,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9782221243596,"""9 hackers combattent un dictateur. Des vies s...","Noa : roman,Levy, Marc","Noa : roman,Levy, Marc",67730,9782221243596
1,1,1,7,16825235,1536557,9789049807245,Zondagskind : alsof opgroeien nog niet lastig ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9789049807245,NaN,Zondagskind : alsof opgroeien nog niet lastig ...,Zondagskind : alsof opgroeien nog niet lastig ...,97553,9789049807245
2,2,2,57,16826563,770735,9789028422087,Bekentenissen van een burger,NaN,NaN,Egy polgár vallomásai,...,NaN,NaN,NaN,NaN,9789028422087,Autobiografisch relaas van een jeugd voor de E...,"Bekentenissen van een burger,Márai, Sándor","Bekentenissen van een burger,Márai, Sándor",6234,9789028422087
3,3,3,72,16828808,539177,"9056175629, 9057593211, 9789056175627, 9789057...",Mijn broer bijvoorbeeld,NaN,NaN,Am Beispiel meines Bruders,...,9057593211,9789056175627,9.789058e+12,NaN,9056175629,Autobiografisch getint relaas waarin een Duits...,"Mijn broer bijvoorbeeld,Timm, Uwe","Mijn broer bijvoorbeeld,Timm, Uwe",63688,"9056175629, 9057593211, 9789056175627, 9789057..."
4,4,4,82,16831632,1922287,9789403185705,De vreemdelinge,NaN,NaN,La straniera,...,NaN,NaN,NaN,NaN,9789403185705,Het levensverhaal van de auteur waarin ze besc...,"De vreemdelinge,Durastanti, Claudia","De vreemdelinge,Durastanti, Claudia",24987,9789403185705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104610,104610,104665,3287452,16880738,583407,"2849020664, 9782849020661",Le nouveau petit Robert : dictionnaire alphabé...,NaN,NaN,NaN,...,9782849020661,NaN,NaN,NaN,2849020664,Découvrez la vie des mots : leur étymologie et...,Le nouveau petit Robert : dictionnaire alphabé...,Le nouveau petit Robert : dictionnaire alphabé...,58307,"2849020664, 9782849020661"
104611,104611,104666,3287470,16882082,579988,"9066484276, 9789066484276",Van Dale groot woordenboek van de Nederlandse ...,NaN,NaN,NaN,...,9789066484276,NaN,NaN,NaN,9066484276,NaN,Van Dale groot woordenboek van de Nederlandse ...,Van Dale groot woordenboek van de Nederlandse ...,88194,"9066484276, 9789066484276"
104612,104612,104667,3287476,17124549,941209,9789066488427,Van Dale beeldwoordenboek Nederlands-Turks,NaN,NaN,The mini visual dictionary,...,NaN,NaN,NaN,NaN,9789066488427,NaN,"Van Dale beeldwoordenboek Nederlands-Turks,Cor...","Van Dale beeldwoordenboek Nederlands-Turks,Cor...",88177,9789066488427
104613,104613,104668,3287480,17025805,1168675,9789460772832,Het Groene Boekje : woordenlijst Nederlandse taal,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9789460772832,NaN,Het Groene Boekje : woordenlijst Nederlandse t...,Het Groene Boekje : woordenlijst Nederlandse t...,41165,9789460772832


In [247]:
df[df['isbn_x'].str.contains('9789021563855')]['book_title']

41449    Miracle morning : 6 gewoontes om je leven succ...
Name: book_title, dtype: object

In [248]:
# Load the embeddings matrix from the .npy file
embeddings_matrix = np.load('embeddings_matrix.npy')

In [249]:
loan_data = pd.read_csv("data_hist_loans.csv", sep=',')
data= loan_data.drop(['Unnamed: 0', 'exem_id', 'transaction_type', 'book_title'], axis=1).reset_index()
data

,index,actor_id,isbn,titelnr,title_author
0,0,1860725,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy"
1,1,1923468,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy"
2,2,1888596,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy"
3,3,1938212,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy"
4,4,1905686,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy"
...,...,...,...,...,...
1296642,1296642,1902133,"9063041063, 9789063041069",196487,"Snooker,Bauwens, Rudy"
1296643,1296643,1904705,9789025307806,1256187,De aarde huilt : de strijd van de indianen om ...
1296644,1296644,1901715,"907656938X, 9789076569383",527114,Sneeuw valt op een zondag in april : gedichten...
1296645,1296645,1870108,9781847394125,902835,"American rust,Meyer, Philipp"


In [253]:
# Create a dictionary mapping book authors to book title IDs
mapping = dict(zip(df['title_author'], df['book_title_id']))

# Map the book_author column in df1 to book_title_id using the mapping
data['book_title_id'] = data['title_author'].map(mapping)
data

,index,actor_id,isbn,titelnr,title_author,book_title_id
0,0,1860725,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy",49762.0
1,1,1923468,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy",49762.0
2,2,1888596,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy",49762.0
3,3,1938212,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy",49762.0
4,4,1905686,9789020679779,1139551,"Ik geef je de zon,Nelson, Jandy",49762.0
...,...,...,...,...,...,...
1296642,1296642,1902133,"9063041063, 9789063041069",196487,"Snooker,Bauwens, Rudy",79055.0
1296643,1296643,1904705,9789025307806,1256187,De aarde huilt : de strijd van de indianen om ...,13612.0
1296644,1296644,1901715,"907656938X, 9789076569383",527114,Sneeuw valt op een zondag in april : gedichten...,78937.0
1296645,1296645,1870108,9781847394125,902835,"American rust,Meyer, Philipp",3616.0


In [226]:
len(data[data['book_title_id'].isna()])

3859

In [254]:
data[data['book_title_id'].isna()]

,index,actor_id,isbn,titelnr,title_author,book_title_id
19260,19260,1972164,NaN,582494,"Wittgenstein,Grayling, A.C.",NaN
19261,19261,1904798,NaN,582494,"Wittgenstein,Grayling, A.C.",NaN
20724,20724,2023167,9789025756628,1142540,Het dierenboek : de encyclopedie van alles wat...,NaN
20725,20725,1972948,9789025756628,1142540,Het dierenboek : de encyclopedie van alles wat...,NaN
20726,20726,1993684,9789025756628,1142540,Het dierenboek : de encyclopedie van alles wat...,NaN
...,...,...,...,...,...,...
1295950,1295950,2014098,9789044821055,1233508,"Ridders,Boshouwers, Susan",NaN
1295951,1295951,1906944,9789044821055,1233508,"Ridders,Boshouwers, Susan",NaN
1296366,1296366,1981253,NaN,811492,"Het raadsel van de Kaaspiramide,Stilton, Geronimo",NaN
1296632,1296632,1930050,NaN,241095,"L'homme de neige,Sand, George",NaN


In [261]:
df[df['book_title'].str.contains('Miracle morning : 6 gewoontes om je leven succesvoller')]

,Unnamed: 0.1,level_0,index,exem_id,titelnr,isbn_x,book_title,deelaand,deeltitel,original_title,...,second_isbn_y,third_isbn_y,forth_isbn_y,book_summary_y,isbn_y,summary,title_author,book_title_c,book_title_id,isbn
41449,41449,41470,733703,16948871,2430316,"9789021563855, 9789021572291",Miracle morning : 6 gewoontes om je leven succ...,NaN,NaN,The miracle morning,...,9789021572291,NaN,NaN,NaN,9789021563855,NaN,Miracle morning : 6 gewoontes om je leven succ...,Miracle morning : 6 gewoontes om je leven succ...,64666,"9789021563855, 9789021572291"
41895,41895,41917,745392,24112754,4122190,9789021572291,Miracle morning : 6 gewoontes om je leven succ...,NaN,NaN,The miracle morning,...,NaN,NaN,NaN,NaN,9789021572291,NaN,Miracle morning : 6 gewoontes om je leven succ...,Miracle morning : 6 gewoontes om je leven succ...,64666,9789021572291


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 187, in _repr_mimebundle_
    dataframe = self._preprocess_dataframe()
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 175, in _preprocess_dataframe
    dataframe = dataframe.reset_index()
  File "/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py", line 6361, in reset_index
    new_obj.insert(
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py", line 4817, in insert
    raise ValueError(f"cannot insert {column}, already exists")
ValueError: cannot insert level_0, already exists
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/google/colab/data_table.py", line 199, in _repr_javascript_module_
    return self._gen_js(self._preprocess_dataframe())
  File "

#####SOLVE

# The ones with missing isbns, create embeddings from sctracth for the 5 ones assign title_id manually. 

In [263]:
data[data['isbn']=='9789021563855']

,index,actor_id,isbn,titelnr,title_author,book_title_id
28006,28006,1963231,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28007,28007,2033205,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28008,28008,1912096,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28009,28009,2060752,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28010,28010,1943008,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28011,28011,1902966,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28012,28012,1925071,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28013,28013,1909848,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28014,28014,1949831,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN
28015,28015,3062708,9789021563855,1219247,Miracle morning : 6 gewoontes om je leven succ...,NaN


In [255]:
# Create a boolean mask for rows in df1 where 'title_id' is NaN
mask = data['book_title_id'].isna()

# Merge the two dataframes on the 'isbn' column for the NaN rows only
merged_df = pd.merge(data.loc[mask], df[['isbn', 'book_title_id']], on='isbn', how='left')

# Fill the missing values in the 'title_id' column of the merged dataframe with the corresponding values from the original dataframe
data.loc[mask[mask==True].index, 'book_title_id'] = merged_df['book_title_id_y'].to_list()
data[data['book_title_id'].isna()]

,index,actor_id,isbn,titelnr,title_author,book_title_id
19260,19260,1972164,NaN,582494,"Wittgenstein,Grayling, A.C.",NaN
19261,19261,1904798,NaN,582494,"Wittgenstein,Grayling, A.C.",NaN
23289,23289,2037324,NaN,1237405,Exercises driving licence B : instant success ...,NaN
23290,23290,1977187,NaN,1237405,Exercises driving licence B : instant success ...,NaN
23291,23291,1987049,NaN,1237405,Exercises driving licence B : instant success ...,NaN
...,...,...,...,...,...,...
1295659,1295659,1871809,NaN,637711,"Kennedy's brein,Mankell, Henning",NaN
1295727,1295727,1900823,NaN,1886487,"Leuven 2030 : roadmap 2025.2035.2050,-",NaN
1296366,1296366,1981253,NaN,811492,"Het raadsel van de Kaaspiramide,Stilton, Geronimo",NaN
1296632,1296632,1930050,NaN,241095,"L'homme de neige,Sand, George",NaN


In [177]:
len(data[data['book_title_id'].isna()]['isbn'].unique())

6

In [173]:
len(data[data['book_title_id'].isna()])

3111

In [174]:
unique_data = data[data['book_title_id'].isna()]['titelnr'].unique()
len(unique_data)

553

In [160]:
data[data['titelnr'] == 955807]

,index,actor_id,isbn,titelnr,title_author,book_title_id
819438,819438,1907166,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819439,819439,1910319,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819440,819440,1900846,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819441,819441,1921442,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819442,819442,1907582,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819443,819443,1905996,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819444,819444,2014304,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
819445,819445,1905590,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
1125674,1125674,2043738,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN
1125675,1125675,2071359,9789056702588,955807,"Op zoek naar jezelf? : basisboek mindfulness,L...",NaN


In [ ]:
data = data.sort_values(['actor_id'], ascending=True)
data

,index,actor_id,titelnr
853489,853489,21195,1310871
108667,108667,21195,1026638
943067,943067,21195,3467257
998142,998142,21195,4136823
403585,403585,21195,1074871
...,...,...,...
333326,333326,3269087,637542
1042753,1042753,3269229,1311818
552554,552554,3269229,1042157
36127,36127,3269229,1168433


In [ ]:
data_in_df = data[-data['titelnr'].isin(df['titelnr'])]
data_in_df

,index,actor_id,titelnr
855355,855355,28960,1053194
448739,448739,28960,1201680
608865,608865,31070,1247581
234208,234208,32993,1069345
278085,278085,54875,1060985
...,...,...,...
589611,589611,3238652,625374
930432,930432,3263055,3355619
1062081,1062081,3266237,956555
1119482,1119482,3266237,3257719


In [ ]:
data_in_df.to_csv('additional_books_without_summary.csv')

In [ ]:
data1 = data.drop_duplicates(['titelnr'])
data1

,index,actor_id,titelnr
853489,853489,21195,1310871
108667,108667,21195,1026638
943067,943067,21195,3467257
998142,998142,21195,4136823
403585,403585,21195,1074871
...,...,...,...
573149,573149,3232013,986486
854371,854371,3232013,1238719
1044882,1044882,3237352,971808
1044881,1044881,3237352,798076


In [ ]:
# Calculating cosine similarities with first book for all the books using 
#embeddings_matrix (took 4secs to run)
import torch
import numpy as np

# Convert embedding_matrix to a PyTorch tensor
embedding_tensor = torch.tensor(embeddings_matrix)

# Compute cosine similarities with the first book
first_book_embeddings = embedding_tensor[df[df['titelnr']==1310871]['index'].iloc[0]]
cosine_similarities = torch.cosine_similarity(first_book_embeddings.unsqueeze(0), embedding_tensor, dim=1)

# Convert cosine_similarities tensor to a NumPy array
cosine_similarities = cosine_similarities.cpu().numpy()
cosine_similarities
# Assign cosine similarities to the 'cosine_similarities_with_first_book' column of the DataFrame
#df['cosine_similarities_with_first_book'] = cosine_similarities

array([0.74763234, 0.89420764, 0.95039842, ..., 0.90835598, 0.81342247,
       0.73390152])

In [ ]:
data1 = data_in_df.drop_duplicates('titelnr')
data1 = data1[:10000]

In [ ]:
#With GPU, 5000 takes 3 mins
#With GPU, 10000 takes 7 mins without crushing
import torch
import numpy as np

# Convert embeddings to a PyTorch tensor
embedding_tensor = torch.tensor(embeddings_matrix).to('cuda')

# Compute cosine similarity matrix
cosine_similarity_matrix = np.zeros((len(data1), 104615))
for i in range(len(data1)):
    titelnr = data1.iloc[i]['titelnr']
    book_i_embeddings = embedding_tensor[df[df['titelnr']==titelnr]['index'].iloc[0]]
    cosine_similarities = torch.cosine_similarity(book_i_embeddings.unsqueeze(0).to('cuda'), embedding_tensor, dim=1)
    cosine_similarity_matrix[i,:] = cosine_similarities.cpu().numpy()
    
# Assign cosine similarity matrix to a new DataFrame
cosine_similarity_df = pd.DataFrame(cosine_similarity_matrix, index=data1['titelnr'])
cosine_similarity_df

,0,1,2,3,4,5,6,7,8,9,...,104605,104606,104607,104608,104609,104610,104611,104612,104613,104614
titelnr,,,,,,,,,,,,,,,,,,,,,
1310871,0.747632,0.894208,0.950398,0.901916,0.965934,0.919412,0.906466,0.925945,0.859593,0.956665,...,0.713688,0.867729,0.773085,0.883854,0.884564,0.728762,0.840907,0.908356,0.813422,0.733902
1026638,0.769395,0.885701,0.952964,0.908510,0.960057,0.892202,0.906352,0.951860,0.863592,0.953935,...,0.724055,0.860795,0.778182,0.897846,0.896206,0.750961,0.840836,0.918518,0.807391,0.756392
3467257,0.736527,0.878709,0.897247,0.874027,0.919988,0.888501,0.878496,0.880257,0.823400,0.906243,...,0.721923,0.841146,0.744718,0.848339,0.842424,0.725571,0.811863,0.860249,0.816730,0.744314
4136823,0.707691,0.890872,0.852293,0.865976,0.879393,0.907964,0.869587,0.833368,0.843036,0.880094,...,0.748054,0.872659,0.730087,0.850253,0.837901,0.720095,0.849162,0.853663,0.819129,0.782766
1074871,0.782408,0.922777,0.860721,0.929653,0.883439,0.920625,0.899237,0.844252,0.904388,0.876106,...,0.833783,0.876085,0.796778,0.878964,0.864054,0.769540,0.845177,0.881827,0.837061,0.880240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004352,0.757781,0.900781,0.953806,0.920602,0.971316,0.916198,0.919717,0.936819,0.863568,0.956191,...,0.722463,0.873422,0.765139,0.901383,0.902152,0.729045,0.848072,0.920672,0.827829,0.751772
3223620,0.925459,0.836674,0.811855,0.858482,0.813649,0.824923,0.847603,0.805038,0.803851,0.805872,...,0.797904,0.775250,0.773989,0.796378,0.782233,0.878571,0.741570,0.792402,0.772134,0.791366
3126049,0.777581,0.944289,0.906139,0.920793,0.927983,0.942531,0.916404,0.871257,0.896892,0.916055,...,0.775677,0.883170,0.773890,0.871389,0.862632,0.752335,0.853676,0.871917,0.850933,0.815184


In [ ]:
cosine_similarity_df.to_csv("cosine_similarity_df_0_10000.csv")

In [ ]:
cosine_similarity_df

,0,1,2,3,4,5,6,7,8,9,...,104605,104606,104607,104608,104609,104610,104611,104612,104613,104614
titelnr,,,,,,,,,,,,,,,,,,,,,
1310871,0.747632,0.894208,0.950398,0.901916,0.965934,0.919412,0.906466,0.925945,0.859593,0.956665,...,0.713688,0.867729,0.773085,0.883854,0.884564,0.728762,0.840907,0.908356,0.813422,0.733902
1026638,0.769395,0.885701,0.952964,0.908510,0.960057,0.892202,0.906352,0.951860,0.863592,0.953935,...,0.724055,0.860795,0.778182,0.897846,0.896206,0.750961,0.840836,0.918518,0.807391,0.756392
3467257,0.736527,0.878709,0.897247,0.874027,0.919988,0.888501,0.878496,0.880257,0.823400,0.906243,...,0.721923,0.841146,0.744718,0.848339,0.842424,0.725571,0.811863,0.860249,0.816730,0.744314
4136823,0.707691,0.890872,0.852293,0.865976,0.879393,0.907964,0.869587,0.833368,0.843036,0.880094,...,0.748054,0.872659,0.730087,0.850253,0.837901,0.720095,0.849162,0.853663,0.819129,0.782766
1074871,0.782408,0.922777,0.860721,0.929653,0.883439,0.920625,0.899237,0.844252,0.904388,0.876106,...,0.833783,0.876085,0.796778,0.878964,0.864054,0.769540,0.845177,0.881827,0.837061,0.880240


In [ ]:
data1.iloc[2]['titelnr']

3467257